In [23]:
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import os
import tabula as tb
from io import BytesIO
import requests as rq


filepath = os.path.realpath(__file__)


def get_google_search_results(query:str):

    # Create a new instance of the Firefox driver
    driver = webdriver.Firefox()

    # Navigate to the specified URL
    url = f"https://www.google.com/search?q={query}"
    driver.get(url)

    # Wait for the page to fully load
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.ID, 'search')))

    # Find the <div id="search"> element
    search_div = driver.find_element(By.ID, 'search')

    # Extract all href attributes from the contained <a> tags
    search_results = search_div.find_elements(By.TAG_NAME, 'a')
    urls = [result.get_attribute('href') for result in search_results]

    driver.quit()  # Close the browser

    print("total ", len(urls), "URLs:\n", urls)
    return urls


def get_html_content(url: str):

    # Create a new instance of the Firefox driver
    driver = webdriver.Firefox()

    # google translate pages
    if "translate.google" in url:
        url = url.partition("?")[2].partition("&u=")[2]

    # Navigate to the specified URL
    driver.get(url)

    # Get the page source (HTML content)
    html_content = driver.page_source

    driver.quit()

    return html_content


def get_pdf_content(url):
    try:
        response = rq.get(url)
        if response.status_code == 200:  # 200 indicates a successful response
            pdf_content = response.content
            return pdf_content
        else:
            print("Error accessing the URL. Status code:", response.status_code)
            return None  # Or raise an exception or handle the error accordingly
    except:
        return None

def extract_tables_with_keywords(html_content, keywords):
    soup = BeautifulSoup(html_content, "html.parser")
    tables = soup.find_all("table")
    keyword_tables = []

    for table in tables:
        for keyword in keywords:
            if keyword in table.text:
                keyword_tables.append(table)
                break

    return keyword_tables


def extract_pdf_tables_with_keywords(pdf_content, keywords):
    tables = []
    try:
        pdf_file = BytesIO(pdf_content)
        dfs = tb.read_pdf(pdf_file, pages='all', multiple_tables=True)
        for df in dfs:
            for keyword in keywords:
                if keyword in df.to_string():
                    tables.append(df)
                    break
    except:
        pass
    return tables

def extract_table_data(table):
    rows = table.find_all("tr")

    data = []
    for row in rows:
        cells = row.find_all("td")
        if cells:
            data.append([cell.text.strip() for cell in cells])

    return data


# ptp list query
query = "ptp+list"
search_results = get_google_search_results(query)

keywords = ["ISIN", "Code", "Ticker", "Symbol", "Security"]

all_table_data = []
urls = []
for index, result in enumerate(search_results, start=1):
    print(f"Result {index}: {result}")
    urls.append(result)
    print("Tables:")
    
    if result.endswith(".pdf"):
        pdf_content = get_pdf_content(result)
        keyword_tables = extract_pdf_tables_with_keywords(pdf_content, keywords)
        for table in keyword_tables:
            all_table_data.extend(table)
            print(pd.DataFrame(table))
            print("--------")
    else:
        html_content = get_html_content(result)
        keyword_tables = extract_tables_with_keywords(html_content, keywords)
        for table in keyword_tables:
            table_data = extract_table_data(table)
            all_table_data.extend(table_data)
            print(pd.DataFrame(table_data))
            print("--------")

    print("--------------------------------------------------")

# Write table data to Excel file
df = pd.DataFrame(all_table_data)
now = datetime.now()
timestamp = now.strftime("%Y-%m-%d_%H-%M-%S")
filename = filepath + "\\" +  "ptp_list_" + timestamp + ".xlsx"


with pd.ExcelWriter(filename, engine='openpyxl') as writer:
    dfURL = pd.DataFrame(urls, columns=["URL"])
    dfURL.to_excel(writer, sheet_name='url', index=False)
    df.to_excel(writer, sheet_name='data', index=False)


print(f"Table data written to {filename}")

total  21 URLs:
 ['https://www.itigerup.com/PTP-list', 'https://www.hsbc.com.hk/content/dam/hsbc/hk/docs/investments/stocks/us-stock-trading/list-of-ptp-instruments.pdf', 'https://www.bocomgroup.com/BankCommSite/upload/infos/202212/01/2652431/20221201163947_List_of_PTP_20220112.pdf', 'http://www.poems.com.hk/en-us/product-and-service/global-securities/usptp/', 'https://translate.google.com/translate?hl=zh-TW&sl=en&u=http://www.poems.com.hk/en-us/product-and-service/global-securities/usptp/&prev=search&pto=aue', 'https://www.upcounsel.com/publicly-traded-partnership', 'https://www.google.com/search?sca_esv=589004769&q=What+are+examples+of+PTP%3F&sa=X&ved=2ahUKEwiStI2dqP-CAxVEIYgKHRtvBAMQzmd6BAgXEAY', 'https://www.investopedia.com/terms/p/ptp.asp', 'https://www.google.com/search?sca_esv=589004769&q=What+is+a+PTP%3F&sa=X&ved=2ahUKEwiStI2dqP-CAxVEIYgKHRtvBAMQzmd6BAgZEAY', 'https://www.irs.gov/individuals/international-taxpayers/publicly-traded-partnerships', 'https://www.google.com/search?